In [2]:
import pulp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import os
import time
import pickle
import warnings
import copy

# --- V7.0 绘图美学配置 ---
warnings.filterwarnings("ignore")
plt.rcParams['font.sans-serif'] = ['Arial', 'SimHei'] 
plt.rcParams['axes.unicode_minus'] = False
try:
    import scienceplots
    plt.style.use(['science', 'no-latex'])
except ImportError:
    sns.set_theme(style="whitegrid", palette="deep", context="paper")

class IP_Solver_Capsule:
    def __init__(self, name="MIP_Model", sense='max'):
        """
        [MCM IP Solver V7.0 - Final Patched]
        Core: Integer Programming (IP) & Mixed Integer Programming (MIP)
        Features: Big-M Logic, Auto-TSP Support, Relaxation Gap, Stability Test, Gephi Export
        """
        self.name = name
        self.timestamp = int(time.time())
        self.sense = pulp.LpMaximize if sense.lower() == 'max' else pulp.LpMinimize
        
        self.prob = pulp.LpProblem(self.name, self.sense)
        
        self.matrix_vars = {}   
        self.single_vars = {}   
        self.binary_vars = []   
        self.aux_vars = {}      # 存储辅助变量 (如 TSP 的 u_i)
        
        self.output_dir = f"./Results_{name}_{self.timestamp}"
        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)

    # ======================================================
    # Phase 0: 握手 (Handshake)
    # ======================================================
    def generate_handshake(self):
        print(f"\n🤝 === 复制以下 Prompt 发送给 AI (V7.0 IP) ===\n")
        print(f"【系统设定】\n我正在使用 `IP_Solver_Capsule` (V7.0)。")
        print(f"目标: {'Maximize' if self.sense == -1 else 'Minimize'} | 输出: `{self.output_dir}`")
        print("\n【API 接口清单】")
        print("1. 矩阵变量: solver.add_var_matrix(rows, cols, name='x', cat='Binary')")
        print("2. 逻辑约束: solver.add_logic_constraint(bin_var, cont_var, logic_type='active_if_1')")
        print("3. TSP回路: solver.add_TSP_subtour_elimination(x_vars, cities) # 自动创建辅助变量")
        print("4. 求解自愈: solver.solve(time_limit=300, gap_rel=0.05)")
        print("5. [MIP核武器] 整数代价: solver.analyze_relaxation_gap()")
        print("6. [MIP核武器] 决策稳定性: solver.analyze_binary_stability()")
        print("7. [可视化] 场景图: visualize_routing / visualize_schedule / visualize_matrix")
        print("8. [可视化] Gephi导出: solver.export_routing_gephi(matrix_name='x')")
        print("9. [交付] 导出: solver.export_results()")
        print("\n【⚠️ 整数规划数学陷阱】")
        print("1. **无影子价格**: MIP 无对偶解。请勿尝试获取 Sensitivity。")
        print("2. **Big-M**: 逻辑约束中 M 若过大可能导致数值不稳定。")

    # ======================================================
    # Phase 1: 建模工厂 (Modeling Factory)
    # ======================================================
    def add_var_matrix(self, rows, cols, name="x", cat='Binary'):
        """ 生成二维变量矩阵并追踪 Binary 变量 """
        vars_dict = pulp.LpVariable.dicts(name, (rows, cols), 0, 1 if cat=='Binary' else None, cat)
        self.matrix_vars[name] = vars_dict
        if cat == 'Binary':
            for r in rows:
                for c in cols:
                    self.binary_vars.append(vars_dict[r][c])
        print(f"✅ 矩阵变量已创建: {name}[{len(rows)}x{len(cols)}] (Type: {cat})")
        return vars_dict

    def add_logic_constraint(self, bin_var, target_var, logic_type='active_if_1', M=1e5):
        """ Big-M 逻辑约束封装 """
        idx = len(self.prob.constraints)
        if logic_type == 'active_if_1':
            self.prob += (target_var <= M * bin_var), f"Logic_Active_{idx}"
        elif logic_type == 'forced_cost':
            self.prob += (target_var >= M * bin_var), f"Logic_FixedCost_{idx}"
        else:
            print(f"⚠️ 未知逻辑类型: {logic_type}")

    def add_TSP_subtour_elimination(self, x_vars, cities):
        """ [Auto] MTZ 约束生成器: 自动创建辅助变量 u_i """
        n = len(cities)
        u_vars = pulp.LpVariable.dicts("u", cities, lowBound=0, upBound=n, cat='Continuous')
        self.aux_vars['u_tsp'] = u_vars
        
        count = 0
        for i in cities:
            if i == cities[0]: continue
            for j in cities:
                if j == cities[0] or i == j: continue
                # MTZ Constraint: u_i - u_j + n*x_ij <= n-1
                if i in x_vars and j in x_vars[i]:
                    self.prob += (u_vars[i] - u_vars[j] + n * x_vars[i][j] <= n - 1), f"Subtour_{i}_{j}"
                    count += 1
        print(f"✅ 已自动生成辅助变量 u 并添加 MTZ 约束: {count} 条")

    # ======================================================
    # Phase 2: 求解与自愈 (Solve & Heal)
    # ======================================================
    def solve(self, solver_name='CBC', time_limit=300, gap_rel=0.05):
        print(f"\n🚀 启动 MIP 求解器 ({solver_name})...")
        print(f"   -> 限制: Time < {time_limit}s, Gap < {gap_rel*100}%")
        if solver_name == 'CBC':
            solver = pulp.PULP_CBC_CMD(timeLimit=time_limit, gapRel=gap_rel, msg=0)
        else:
            solver = pulp.PULP_CBC_CMD(msg=0)
        try: status = self.prob.solve(solver)
        except Exception as e: print(f"❌ 求解器错误: {e}"); return
        
        status_str = pulp.LpStatus[status]
        print(f"📋 求解状态: {status_str}")
        if status_str in ['Infeasible', 'Unbounded']:
            print("🚨 警告: 模型无解或无界！启动自动诊断...")
            self._diagnose_slack()
        else:
            print(f"💎 最优目标值: {pulp.value(self.prob.objective)}")
            try:
                with open(f"{self.output_dir}/model_checkpoint.pkl", 'wb') as f:
                    pickle.dump(self.prob, f)
            except: pass

    def _diagnose_slack(self):
        print("\n🔍 --- MIP 诊断建议 ---")
        print("1. **互斥检查**: sum(x) == 1 是否因资源不足无法满足？")
        print("2. **Big-M**: M 是否过小截断了可行域？")
        print("3. **建议**: 尝试将 Integer 改为 Continuous 求解，定位是否为整数性导致的无解。")

    # ======================================================
    # Phase 3: 深度分析 (Deep Analysis)
    # ======================================================
    def analyze_relaxation_gap(self):
        print(f"\n🧩 启动整数代价分析 (Relaxation Gap)...")
        if self.prob.status != 1: return
        ip_obj = pulp.value(self.prob.objective)
        original_cats = {}
        for v in self.prob.variables():
            original_cats[v.name] = v.cat
            v.cat = pulp.LpContinuous
        self.prob.solve(pulp.PULP_CBC_CMD(msg=0))
        lp_obj = pulp.value(self.prob.objective)
        for v in self.prob.variables(): v.cat = original_cats[v.name]
        self.prob.solve(pulp.PULP_CBC_CMD(msg=0)) # Restore
        if lp_obj is None: return
        gap = abs(ip_obj - lp_obj)
        gap_percent = (gap / abs(ip_obj)) * 100 if ip_obj != 0 else 0
        print(f"   -> IP (整数) 目标值: {ip_obj}")
        print(f"   -> LP (松弛) 目标值: {lp_obj}")
        print(f"   -> 整数代价 (Cost of Integrity): {gap:.4f} ({gap_percent:.2f}%)")
        print("   💡 解释: 为了满足离散约束，我们牺牲了 {:.2f}% 的理论最优效益。".format(gap_percent))
        with open(f"{self.output_dir}/Gap_Analysis.txt", "w") as f:
            f.write(f"IP_Obj: {ip_obj}\nLP_Obj: {lp_obj}\nGap: {gap_percent:.2f}%\n")

    def analyze_binary_stability(self, perturb_range=0.1, runs=10):
        print(f"\n🌪️ 启动决策稳定性分析 (Runs={runs}, Perturb=±{perturb_range:.0%})...")
        if not self.binary_vars: return
        base_sol = {v.name: v.varValue for v in self.binary_vars}
        flip_counts = {v.name: 0 for v in self.binary_vars}
        original_objective = self.prob.objective
        try: coeffs = original_objective.to_dict()
        except: return
        valid_runs = 0
        for r in range(runs):
            new_obj = 0
            for v, c in coeffs.items():
                new_obj += c * (1 + np.random.uniform(-perturb_range, perturb_range)) * v
            self.prob.setObjective(new_obj)
            self.prob.solve(pulp.PULP_CBC_CMD(msg=0))
            if self.prob.status == 1:
                valid_runs += 1
                for v in self.binary_vars:
                    if abs(v.varValue - base_sol[v.name]) > 0.5:
                        flip_counts[v.name] += 1
            print(".", end="")
        print(f" 完成! (Valid Runs: {valid_runs})")
        self.prob.setObjective(original_objective) 
        self.prob.solve(pulp.PULP_CBC_CMD(msg=0)) 
        flip_rates = [flip_counts[v.name]/valid_runs for v in self.binary_vars if valid_runs > 0]
        df_flip = pd.DataFrame({'Variable': [v.name for v in self.binary_vars], 'Flip_Rate': flip_rates})
        df_active = df_flip[df_flip['Flip_Rate'] > 0].sort_values('Flip_Rate', ascending=False).head(20)
        if not df_active.empty:
            plt.figure(figsize=(10, 6))
            sns.barplot(x='Flip_Rate', y='Variable', data=df_active, palette='Reds')
            plt.title('Stability Analysis: Decision Flip Rate')
            plt.tight_layout()
            plt.savefig(f"{self.output_dir}/Stability_Heatmap.svg")
            print(f"✅ 稳定性热力图已保存。")
        else:
            print("✅ 决策极其稳健。")

    # ======================================================
    # Phase 4: 可视化与 Gephi (Visualization)
    # ======================================================
    def visualize_routing(self, from_nodes, to_nodes, active_matrix_name='x', pos_dict=None):
        print(f"\n🗺️ 绘制路径规划图...")
        if active_matrix_name not in self.matrix_vars: return
        G = nx.DiGraph()
        vars_dict = self.matrix_vars[active_matrix_name]
        edge_count = 0
        G.add_nodes_from(set(list(from_nodes) + list(to_nodes)))
        for i in from_nodes:
            for j in to_nodes:
                if i == j: continue
                if i in vars_dict and j in vars_dict[i]:
                    val = pulp.value(vars_dict[i][j])
                    if val and val > 0.9: 
                        G.add_edge(i, j)
                        edge_count += 1
        if edge_count == 0: return
        plt.figure(figsize=(10, 8))
        pos = pos_dict if pos_dict else nx.spring_layout(G, k=0.5, seed=42)
        nx.draw_networkx(G, pos, node_size=500, node_color='skyblue', with_labels=True, arrowsize=20)
        plt.title(f'Optimal Routing Plan ({edge_count} Segments)')
        plt.axis('off')
        plt.savefig(f"{self.output_dir}/Routing_Graph.svg")

    def visualize_schedule(self, df_schedule):
        """ 甘特图: ['Task', 'Start', 'End', 'Resource'] """
        print(f"\n📅 绘制甘特图...")
        if df_schedule.empty: return
        df = df_schedule.copy()
        df['Duration'] = df['End'] - df['Start']
        plt.figure(figsize=(12, 6))
        resources = df['Resource'].unique()
        colors = sns.color_palette("hls", len(resources))
        color_map = dict(zip(resources, colors))
        
        # 将 Task 映射为 Y 轴坐标，防止中文/字符串报错
        tasks = df['Task'].unique()
        task_map = {t: i for i, t in enumerate(tasks)}
        
        for idx, row in df.iterrows():
            y_pos = task_map[row['Task']]
            plt.barh(y=y_pos, width=row['Duration'], left=row['Start'], 
                     color=color_map[row['Resource']], alpha=0.8, edgecolor='black')
            plt.text(row['Start'] + row['Duration']/2, y_pos, str(row['Resource']), 
                     ha='center', va='center', color='white', fontsize=8)
            
        plt.yticks(list(task_map.values()), list(task_map.keys()))
        plt.xlabel("Time"); plt.ylabel("Task")
        plt.title("Optimization Schedule")
        plt.grid(axis='x', alpha=0.3)
        plt.tight_layout()
        plt.savefig(f"{self.output_dir}/Gantt_Chart.svg")

    def visualize_matrix(self, matrix_name):
        print(f"\n▦ 绘制分配矩阵热力图 ({matrix_name})...")
        if matrix_name not in self.matrix_vars: return
        vars_dict = self.matrix_vars[matrix_name]
        rows = list(vars_dict.keys())
        cols = list(vars_dict[rows[0]].keys())
        data = [[pulp.value(vars_dict[r][c]) for c in cols] for r in rows]
        plt.figure(figsize=(10, 8))
        sns.heatmap(data, cmap="Blues", xticklabels=cols, yticklabels=rows, linewidths=.5, cbar=False)
        plt.title(f'Assignment Matrix: {matrix_name}')
        plt.savefig(f"{self.output_dir}/Matrix_Heatmap.svg")

    def export_routing_gephi(self, matrix_name='x'):
        """ [New] Gephi 路由导出协议 """
        print(f"\n🕸️ 正在导出路由网络 Gephi 数据 (Matrix: {matrix_name})...")
        if matrix_name not in self.matrix_vars: return
        vars_dict = self.matrix_vars[matrix_name]
        edges = []
        for i in vars_dict:
            for j in vars_dict[i]:
                val = pulp.value(vars_dict[i][j])
                if val and val > 0.9:
                    edges.append({'Source': i, 'Target': j, 'Weight': 1, 'Type': 'Directed'})
        if not edges: return
        df_edges = pd.DataFrame(edges)
        save_path = f"{self.output_dir}/Routing_Flow_Gephi.csv"
        df_edges.to_csv(save_path, index=False)
        print(f"✅ 已导出 Gephi 边表: {save_path} (含 {len(edges)} 条路径)")

    def export_results(self):
        print(f"\n📦 === 正在打包交付物至 {self.output_dir} === ")
        data = []
        for v in self.prob.variables():
            val = v.varValue
            if val and abs(val) > 1e-5:
                data.append({'Variable': v.name, 'Value': val})
        df_res = pd.DataFrame(data)
        df_res.to_excel(f"{self.output_dir}/Solution.xlsx", index=False)
        
        obj_val = pulp.value(self.prob.objective)
        status = pulp.LpStatus[self.prob.status]
        report = f"# IP Report: {self.name}\n\nStatus: {status}\nObjective: {obj_val}\n"
        report += "## Analysis\n- Check `Gap_Analysis.txt` for cost of integrity.\n"
        report += "- Check `Stability_Heatmap.svg` for decision robustness.\n"
        if os.path.exists(f"{self.output_dir}/Routing_Flow_Gephi.csv"):
            report += "- Check `Routing_Flow_Gephi.csv` for network visualization.\n"
        with open(f"{self.output_dir}/Report.md", "w", encoding='utf-8') as f: f.write(report)
        
        if not df_res.empty:
            df_top = df_res.head(15)
            tex = "\\begin{tabular}{lr} Variable & Value \\\\ \\midrule\n"
            for _, r in df_top.iterrows(): tex += f"{r['Variable'].replace('_','\\_')} & {r['Value']} \\\\\n"
            tex += "\\end{tabular}"
            with open(f"{self.output_dir}/Variables.tex", "w") as f: f.write(tex)
        print(f"✅ 全套结果已导出 (Excel, Report, SVG, Gephi)。")

In [4]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

# ==========================================
# 🛑 Phase 0: Initialization & Pre-processing
# ==========================================

# 1. 初始化求解器
# ------------------------------------------
solver = IP_Solver_Capsule(name="GreenGrid", sense='min')
solver.generate_handshake()

print(f"\n[System] 正在加载并清洗数据...")

# 2. 数据加载与清洗 (Data Cleaning)
# ------------------------------------------
# 假设本地存在 CSV 文件，若不存在请替换为实际路径
try:
    df_sites = pd.read_csv('City_Sites.csv')
    df_demands = pd.read_csv('City_Demands.csv')
except FileNotFoundError:
    # 🚨 为演示代码运行，若无文件则生成模拟数据 (实际运行时请删除此块)
    print("⚠️ 未找到文件，生成模拟数据用于演示逻辑...")
    np.random.seed(42)
    df_sites = pd.DataFrame({
        'Site_ID': range(10),
        'X': np.random.rand(10) * 50, 'Y': np.random.rand(10) * 50,
        'Grid_Capacity_kW': [np.nan, 100, 200, np.nan] + [150]*6
    })
    df_demands = pd.DataFrame({
        'Demand_ID': range(50),
        'X': np.random.rand(50) * 50, 'Y': np.random.rand(50) * 50,
        'Daily_Traffic': np.random.choice([0, -5, 10, 50, 100], 50)
    })

# --- 清洗 Sites ---
raw_sites_count = len(df_sites)
median_cap = df_sites['Grid_Capacity_kW'].median()
df_sites['Grid_Capacity_kW'].fillna(median_cap, inplace=True)

# --- 清洗 Demands ---
raw_demands_count = len(df_demands)
df_demands_clean = df_demands[df_demands['Daily_Traffic'] > 0].copy().reset_index(drop=True)

# --- 验证输出 ---
print(f"📊 [Data Quality Check]")
print(f"   - Sites:   {raw_sites_count} -> {len(df_sites)} (NaN filled with {median_cap:.1f})")
print(f"   - Demands: {raw_demands_count} -> {len(df_demands_clean)} (Removed {raw_demands_count - len(df_demands_clean)} invalid rows)")

# 3. 空间剪枝 (Spatial Pruning)
# ------------------------------------------
# 假设坐标列名为 'X' 和 'Y'
coords_demands = df_demands_clean[['X', 'Y']].values
coords_sites = df_sites[['X', 'Y']].values

# 计算距离矩阵 (Rows: Demands, Cols: Sites)
dist_matrix = cdist(coords_demands, coords_sites, metric='euclidean')

# 筛选 <= 15 的连接
threshold = 15
valid_indices = np.where(dist_matrix <= threshold)
valid_pairs = []

# 构建稀疏索引列表: [(Demand_ID, Site_ID, Distance), ...]
for r, c in zip(valid_indices[0], valid_indices[1]):
    d_id = df_demands_clean.iloc[r]['Demand_ID']
    s_id = df_sites.iloc[c]['Site_ID']
    dist = dist_matrix[r][c]
    valid_pairs.append((d_id, s_id, dist))

# --- 剪枝效果统计 ---
full_permutation = len(df_demands_clean) * len(df_sites)
valid_count = len(valid_pairs)
reduction_rate = (1 - valid_count / full_permutation) * 100

print(f"\n✂️ [Spatial Pruning Result]")
print(f"   - Threshold: Distance <= {threshold}")
print(f"   - Valid Pairs: {valid_count} (out of {full_permutation})")
print(f"   - Optimization: Reduced variable space by {reduction_rate:.2f}%")

# 保存预处理后的有效对，供 Phase 1 使用
# format: {(d_id, s_id): dist}
valid_dist_dict = {(p[0], p[1]): p[2] for p in valid_pairs}


🤝 === 复制以下 Prompt 发送给 AI (V7.0 IP) ===

【系统设定】
我正在使用 `IP_Solver_Capsule` (V7.0)。
目标: Minimize | 输出: `./Results_GreenGrid_1765977725`

【API 接口清单】
1. 矩阵变量: solver.add_var_matrix(rows, cols, name='x', cat='Binary')
2. 逻辑约束: solver.add_logic_constraint(bin_var, cont_var, logic_type='active_if_1')
3. TSP回路: solver.add_TSP_subtour_elimination(x_vars, cities) # 自动创建辅助变量
4. 求解自愈: solver.solve(time_limit=300, gap_rel=0.05)
5. [MIP核武器] 整数代价: solver.analyze_relaxation_gap()
6. [MIP核武器] 决策稳定性: solver.analyze_binary_stability()
7. [可视化] 场景图: visualize_routing / visualize_schedule / visualize_matrix
8. [可视化] Gephi导出: solver.export_routing_gephi(matrix_name='x')
9. [交付] 导出: solver.export_results()

【⚠️ 整数规划数学陷阱】
1. **无影子价格**: MIP 无对偶解。请勿尝试获取 Sensitivity。
2. **Big-M**: 逻辑约束中 M 若过大可能导致数值不稳定。

[System] 正在加载并清洗数据...
📊 [Data Quality Check]
   - Sites:   20 -> 20 (NaN filled with 637.0)
   - Demands: 50 -> 47 (Removed 3 invalid rows)

✂️ [Spatial Pruning Result]
   - Threshold: Distance <= 15
   - Valid P

In [5]:
# ==========================================
# 🏗️ Phase 1: 变量定义 (Variable Definition)
# ==========================================

# 1. 准备索引集合
# ------------------------------------------
sites_ids = df_sites['Site_ID'].tolist()
# 提取稀疏键值对: (Demand_ID, Site_ID)
y_keys = [(p[0], p[1]) for p in valid_pairs] 

print(f"\n[Modeling] 正在构建变量体系...")

# 2. 创建变量 (使用底层 Pulp 方法以适配稀疏结构)
# ------------------------------------------

# --- A. 选址变量 z (Binary, 1D) ---
# z[j] = 1 if Site j is built
z = pulp.LpVariable.dicts("z", sites_ids, cat='Binary')
solver.single_vars['z'] = z
# [关键]: 手动注册到 V7.0 的 binary_vars 以启用稳定性分析
solver.binary_vars.extend(z.values())
print(f"✅ 变量创建: z (选址状态) | 维度: {len(z)} | Type: Binary")

# --- B. 配置变量 n (Integer, 1D) ---
# n_fast[j], n_slow[j]: Number of chargers at Site j
n_fast = pulp.LpVariable.dicts("n_fast", sites_ids, lowBound=0, upBound=50, cat='Integer')
n_slow = pulp.LpVariable.dicts("n_slow", sites_ids, lowBound=0, upBound=50, cat='Integer')
solver.single_vars['n_fast'] = n_fast
solver.single_vars['n_slow'] = n_slow
print(f"✅ 变量创建: n (充电桩数量) | 维度: {len(n_fast)}x2 | Type: Integer (0-50)")

# --- C. 指派变量 y (Binary, Sparse 2D) ---
# y[(i,j)] = 1 if Demand i assigned to Site j
# 仅针对 valid_pairs 创建，而非 N*M 全连接
y = pulp.LpVariable.dicts("y", y_keys, cat='Binary')
solver.matrix_vars['y'] = y  # 注册以便可视化调用
# [关键]: 手动注册到 V7.0 的 binary_vars
solver.binary_vars.extend(y.values())
print(f"✅ 变量创建: y (需求指派) | 维度: {len(y)} (Sparse) | Type: Binary")

# 3. 完整性自检
# ------------------------------------------
print(f"\n🛡️ [V7.0 Integrity Check]")
print(f"   -> 这里的 y 变量仅包含了 {len(y)} 个有效连接 (基于 Phase 0 剪枝)。")
print(f"   -> 已追踪 Binary 变量总数: {len(solver.binary_vars)} (用于 Phase 4 稳定性测试)")


[Modeling] 正在构建变量体系...
✅ 变量创建: z (选址状态) | 维度: 20 | Type: Binary
✅ 变量创建: n (充电桩数量) | 维度: 20x2 | Type: Integer (0-50)
✅ 变量创建: y (需求指派) | 维度: 58 (Sparse) | Type: Binary

🛡️ [V7.0 Integrity Check]
   -> 这里的 y 变量仅包含了 58 个有效连接 (基于 Phase 0 剪枝)。
   -> 已追踪 Binary 变量总数: 78 (用于 Phase 4 稳定性测试)


In [6]:
# ==========================================
# 🧠 Phase 2: 逻辑注入与约束构建
# ==========================================
import pulp 

print(f"\n[Constraints] 正在注入业务逻辑与物理约束...")

# ------------------------------------------
# 0. [专家修正] 刚性服务约束 (Mandatory Service)
# ------------------------------------------
# 逻辑: 每个需求点 i 必须且只能连接到一个站点 j
# 防止 Min Cost 目标导致“全0解”
count_service = 0
# 预先构建 i -> [j1, j2...] 的映射以加速索引
i_to_j_map = {}
for (i, j) in solver.matrix_vars['y']:
    if i not in i_to_j_map: i_to_j_map[i] = []
    i_to_j_map[i].append(j)

# 假设需求ID列表为 demand_ids (从 df_demands 获取)
demand_ids = df_demands_clean['Demand_ID'].unique()

for i in demand_ids:
    if i in i_to_j_map:
        # 约束: sum(y_ij for all j) == 1
        solver.prob += (pulp.lpSum(solver.matrix_vars['y'][(i, j)] for j in i_to_j_map[i]) == 1, f"Must_Serve_{i}")
        count_service += 1
print(f"✅ [Auto-Correction] 已添加刚性服务约束: {count_service} 条 (必须满足需求)")

# ------------------------------------------
# 1. 选址依赖 (Big-M Automation)
# ------------------------------------------
# 逻辑: 只有 z[j]=1 (建站)，才能有流量 y[i][j]=1
count_bigm = 0
for (i, j), y_var in solver.matrix_vars['y'].items():
    # 调用 V7.0 封装接口: y <= M * z (此处 M=1 即可)
    solver.add_logic_constraint(bin_var=solver.single_vars['z'][j], 
                                target_var=y_var, 
                                logic_type='active_if_1', 
                                M=1)
    count_bigm += 1
print(f"✅ 选址依赖 (Big-M): {count_bigm} 条 (y <= z)")

# ------------------------------------------
# 2. 装机依赖与电网硬限 (Physical & Grid)
# ------------------------------------------
count_cap = 0
# 遍历 df_sites 获取每个站点的参数
for idx, row in df_sites.iterrows():
    j = row['Site_ID']
    # 获取变量
    z_var = solver.single_vars['z'][j]
    nf_var = solver.single_vars['n_fast'][j]
    ns_var = solver.single_vars['n_slow'][j]
    grid_cap = row['Grid_Capacity_kW']
    
    # 2.1 物理存在约束: 不建站(z=0)则桩数必须为0
    # n_fast + n_slow <= 50 * z
    solver.prob += (nf_var + ns_var <= 50 * z_var, f"Phy_Exist_{j}")
    
    # 2.2 电网功率硬限
    # 120*fast + 7*slow <= GridCapacity
    solver.prob += (120 * nf_var + 7 * ns_var <= grid_cap, f"Grid_Limit_{j}")
    count_cap += 1
print(f"✅ 物理与电网约束: {count_cap} 组 (Capacity & Existence)")

# ------------------------------------------
# 3. 区域政策 (Data-Driven Logic)
# ------------------------------------------
count_zone = 0
for idx, row in df_sites.iterrows():
    if row['Zone_Type'] == 'Commercial':
        j = row['Site_ID']
        # 商业区策略: 快充数 >= 慢充数
        nf_var = solver.single_vars['n_fast'][j]
        ns_var = solver.single_vars['n_slow'][j]
        solver.prob += (nf_var >= ns_var, f"Policy_Comm_{j}")
        count_zone += 1
print(f"✅ 区域政策约束: {count_zone} 条 (Commercial Zone Preference)")

# ------------------------------------------
# 4. 供需平衡 (Capacity vs Load)
# ------------------------------------------
# 逻辑: 3 * (n_fast + n_slow) >= sum(Traffic * y)
# 构建 j -> [i1, i2...] 映射
j_to_i_map = {}
for (i, j) in solver.matrix_vars['y']:
    if j not in j_to_i_map: j_to_i_map[j] = []
    j_to_i_map[j].append(i)

# 流量查找表
traffic_map = dict(zip(df_demands_clean['Demand_ID'], df_demands_clean['Daily_Traffic']))

count_bal = 0
for j in sites_ids:
    if j in j_to_i_map:
        # 计算该站点被分配的总流量
        assigned_traffic = pulp.lpSum(solver.matrix_vars['y'][(i, j)] * traffic_map[i] for i in j_to_i_map[j])
        # 计算该站点的服务能力
        service_cap = 3 * (solver.single_vars['n_fast'][j] + solver.single_vars['n_slow'][j])
        
        solver.prob += (service_cap >= assigned_traffic, f"Supply_Bal_{j}")
        count_bal += 1
print(f"✅ 供需平衡约束: {count_bal} 条 (Service >= Traffic)")

# ------------------------------------------
# 5. 目标函数 (Objective: Min Cost)
# ------------------------------------------
# Cost 1: Land (z * LandCost)
# 假设 df_sites 有 Land_Cost 列，若无请用 100 替代
land_cost_map = dict(zip(df_sites['Site_ID'], df_sites.get('Land_Cost', 100)))
obj_land = pulp.lpSum(solver.single_vars['z'][j] * land_cost_map[j] for j in sites_ids)

# Cost 2: Device (Fast*10 + Slow*2)
obj_device = pulp.lpSum(solver.single_vars['n_fast'][j] * 10 + solver.single_vars['n_slow'][j] * 2 for j in sites_ids)

# Cost 3: Access (Dist * y * 0.1)
# valid_dist_dict 来自 Phase 0
obj_access = pulp.lpSum(valid_dist_dict[(i,j)] * solver.matrix_vars['y'][(i,j)] * 0.1 for (i,j) in solver.matrix_vars['y'])

# Set Objective
solver.prob += (obj_land + obj_device + obj_access)
print(f"\n🎯 目标函数设定完毕 (Min Total Cost)")


[Constraints] 正在注入业务逻辑与物理约束...
✅ [Auto-Correction] 已添加刚性服务约束: 30 条 (必须满足需求)
✅ 选址依赖 (Big-M): 58 条 (y <= z)
✅ 物理与电网约束: 20 组 (Capacity & Existence)
✅ 区域政策约束: 9 条 (Commercial Zone Preference)
✅ 供需平衡约束: 19 条 (Service >= Traffic)

🎯 目标函数设定完毕 (Min Total Cost)


In [7]:
# ==========================================
# 🚀 Phase 3: 求解与结果解析 (Solve & Parse)
# ==========================================
import pandas as pd
import pulp

print(f"\n[Solving] 启动求解引擎 (TimeLimit=60s, Gap=5%)...")

# 1. 执行求解
# ------------------------------------------
solver.solve(time_limit=60, gap_rel=0.05)

# 2. 结果解析与报表生成
# ------------------------------------------
if solver.prob.status == 1: # Optimal or Feasible
    print(f"\n📊 正在生成建设方案表...")
    
    # 准备流量查找表 (用于计算 Served_Traffic)
    traffic_map = dict(zip(df_demands_clean['Demand_ID'], df_demands_clean['Daily_Traffic']))
    
    construction_plan = []
    
    # 遍历所有站点，筛选 z=1 的站点
    for idx, row in df_sites.iterrows():
        j = row['Site_ID']
        # 获取选址变量值
        z_val = pulp.value(solver.single_vars['z'][j])
        
        # 仅处理被选中的站点 (z > 0.5)
        if z_val and z_val > 0.5:
            # 获取配置变量值
            n_fast_val = pulp.value(solver.single_vars['n_fast'][j])
            n_slow_val = pulp.value(solver.single_vars['n_slow'][j])
            
            # 计算该站点服务的总流量 (遍历指派变量 y)
            served_traffic = 0
            for (i, s_id), y_var in solver.matrix_vars['y'].items():
                if s_id == j:
                    y_val = pulp.value(y_var)
                    if y_val and y_val > 0.5:
                        served_traffic += traffic_map.get(i, 0)
            
            # 记录结果
            construction_plan.append({
                'Site_ID': int(j),
                'Zone_Type': row.get('Zone_Type', 'Unknown'),
                'Fast_Chargers': int(n_fast_val),
                'Slow_Chargers': int(n_slow_val),
                'Served_Traffic': round(served_traffic, 1)
            })
    
    # 转为 DataFrame 并美化输出
    df_plan = pd.DataFrame(construction_plan)
    
    if not df_plan.empty:
        print("\n🏆 [GreenGrid 最终建设方案]")
        # 格式化输出: 居中对齐或制表符
        print(df_plan.to_string(index=False))
        
        # 简单统计
        total_fast = df_plan['Fast_Chargers'].sum()
        total_slow = df_plan['Slow_Chargers'].sum()
        print(f"\n📝 汇总统计:")
        print(f"   - 总建设站点数: {len(df_plan)}")
        print(f"   - 快充桩总数: {total_fast}")
        print(f"   - 慢充桩总数: {total_slow}")
    else:
        print("⚠️ 警告: 求解器返回了空方案 (所有 z=0)。请检查 Phase 2 的强制服务约束是否生效。")

else:
    print(f"❌ 求解失败或无解。状态: {pulp.LpStatus[solver.prob.status]}")
    # 触发诊断
    solver._diagnose_slack()


[Solving] 启动求解引擎 (TimeLimit=60s, Gap=5%)...

🚀 启动 MIP 求解器 (CBC)...
   -> 限制: Time < 60s, Gap < 5.0%
📋 求解状态: Infeasible
🚨 警告: 模型无解或无界！启动自动诊断...

🔍 --- MIP 诊断建议 ---
1. **互斥检查**: sum(x) == 1 是否因资源不足无法满足？
2. **Big-M**: M 是否过小截断了可行域？
3. **建议**: 尝试将 Integer 改为 Continuous 求解，定位是否为整数性导致的无解。
❌ 求解失败或无解。状态: Infeasible

🔍 --- MIP 诊断建议 ---
1. **互斥检查**: sum(x) == 1 是否因资源不足无法满足？
2. **Big-M**: M 是否过小截断了可行域？
3. **建议**: 尝试将 Integer 改为 Continuous 求解，定位是否为整数性导致的无解。


In [8]:
# ==========================================
# 🔍 Phase 3.5: 法医诊断 (Forensic Analysis)
# ==========================================
print(f"\n🕵️‍♂️ 启动死因调查程序...")

# 1. 检查孤岛危机 (Orphans)
# ------------------------------------------
orphaned_demands = []
for i in demand_ids: # demand_ids from Phase 2
    if i not in i_to_j_map or len(i_to_j_map[i]) == 0:
        orphaned_demands.append(i)

if orphaned_demands:
    print(f"❌ [致命原因] 发现 {len(orphaned_demands)} 个孤岛需求点 (无可用站点):")
    print(f"   -> IDs: {orphaned_demands[:10]} ...")
    print("   💡 建议: 返回 Phase 0，将距离阈值 (threshold) 从 15 调大到 30 或 50。")
else:
    print(f"✅ 连通性检查通过: 所有需求点至少有一个候选站点。")

# 2. 检查容量陷阱 (Capacity Traps)
# ------------------------------------------
# 逻辑: 检查每个需求点，其候选站点的“理论最大服务能力”是否小于“实际交通需求”
capacity_traps = []

for i in demand_ids:
    if i in orphaned_demands: continue
    
    required_traffic = traffic_map[i]
    max_possible_service = 0
    
    candidates = i_to_j_map[i]
    for j in candidates:
        # 获取该站点的最大物理能力
        row = df_sites[df_sites['Site_ID'] == j].iloc[0]
        cap = row['Grid_Capacity_kW']
        
        # 估算该站点的最大服务量 (Max Service)
        # 限制1: 物理位 50个 -> max 50 * 3 = 150
        # 限制2: 电网功率 -> max (cap / 7) * 3 (全慢充最省电，算极限)
        # 注意: 这里只是粗估上限，用于诊断
        limit_slots = 50
        limit_grid = int(cap / 7) 
        max_chargers = min(limit_slots, limit_grid)
        max_service_j = max_chargers * 3 
        
        max_possible_service += max_service_j # 假设所有候选站点都全力服务它(虽然不可能同时，但这是上限)

    if max_possible_service < required_traffic:
        capacity_traps.append({
            'Demand_ID': i, 
            'Required': required_traffic, 
            'Max_Supply': max_possible_service,
            'Candidates': candidates
        })

if capacity_traps:
    print(f"❌ [致命原因] 发现 {len(capacity_traps)} 个需求点遭遇容量死锁 (周围站点电不够):")
    df_trap = pd.DataFrame(capacity_traps)
    print(df_trap.head())
    print("   💡 建议: 检查 City_Demands 的流量值是否过大，或 Grid_Capacity 是否过小。")
else:
    print(f"✅ 容量理论检查通过: 局部容量未见明显死锁。")

if not orphaned_demands and not capacity_traps:
    print("⚠️ 未发现明显硬伤。可能是 '区域政策(Commercial)' 或 '供需平衡' 的组合约束导致冲突。")
    print("   💡 建议: 尝试注释掉 Phase 2 的 '区域政策' 约束，看看是否可行。")


🕵️‍♂️ 启动死因调查程序...
❌ [致命原因] 发现 17 个孤岛需求点 (无可用站点):
   -> IDs: ['D05', 'D07', 'D10', 'D12', 'D21', 'D22', 'D25', 'D27', 'D28', 'D30'] ...
   💡 建议: 返回 Phase 0，将距离阈值 (threshold) 从 15 调大到 30 或 50。
✅ 容量理论检查通过: 局部容量未见明显死锁。


In [9]:
# ==========================================
# 🚑 System Reboot: Hotfix Patch v1.1
# ==========================================
# 目标: 修复孤岛问题 (Threshold 15 -> 35) -> 重建模型 -> 求解
# ==========================================

import pulp
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

print(f"\n🔄 [System Reboot] 正在应用修复补丁 (Threshold=35km)...")

# 1. 重置求解器 (Clean Slate)
# ------------------------------------------
solver = IP_Solver_Capsule(name="GreenGrid_Fixed", sense='min')

# 2. Phase 0: 宽松版空间剪枝 (Relaxed Pruning)
# ------------------------------------------
coords_demands = df_demands_clean[['X', 'Y']].values
coords_sites = df_sites[['X', 'Y']].values
dist_matrix = cdist(coords_demands, coords_sites, metric='euclidean')

# [修复动作] 扩大阈值至 35km
new_threshold = 35 
valid_indices = np.where(dist_matrix <= new_threshold)
valid_pairs = []
valid_dist_dict = {}

for r, c in zip(valid_indices[0], valid_indices[1]):
    d_id = df_demands_clean.iloc[r]['Demand_ID']
    s_id = df_sites.iloc[c]['Site_ID']
    dist = dist_matrix[r][c]
    valid_pairs.append((d_id, s_id, dist))
    valid_dist_dict[(d_id, s_id)] = dist

print(f"✅ 空间索引重构完成: {len(valid_pairs)} valid pairs (Threshold={new_threshold})")

# [自检] 再次检查孤岛
connected_demands = set([p[0] for p in valid_pairs])
all_demands = set(df_demands_clean['Demand_ID'])
orphans = all_demands - connected_demands
if orphans:
    print(f"❌ 警告: 仍有 {len(orphans)} 个孤岛! (建议改为 50km)")
else:
    print(f"✅ 孤岛危机解除: 所有需求点均已覆盖。")

# 3. Phase 1: 变量重建 (Variable Re-definition)
# ------------------------------------------
sites_ids = df_sites['Site_ID'].tolist()
y_keys = [(p[0], p[1]) for p in valid_pairs]

# z (Sites), n (Config)
z = pulp.LpVariable.dicts("z", sites_ids, cat='Binary')
n_fast = pulp.LpVariable.dicts("n_fast", sites_ids, lowBound=0, upBound=50, cat='Integer')
n_slow = pulp.LpVariable.dicts("n_slow", sites_ids, lowBound=0, upBound=50, cat='Integer')

solver.single_vars.update({'z': z, 'n_fast': n_fast, 'n_slow': n_slow})
solver.binary_vars.extend(z.values())

# y (Assign) - Sparse
y = pulp.LpVariable.dicts("y", y_keys, cat='Binary')
solver.matrix_vars['y'] = y
solver.binary_vars.extend(y.values())

# 4. Phase 2: 约束重注 (Logic Injection)
# ------------------------------------------
# 4.1 刚性服务 (Must Serve)
i_to_j_map = {}
for (i, j) in y_keys:
    if i not in i_to_j_map: i_to_j_map[i] = []
    i_to_j_map[i].append(j)

for i in all_demands:
    if i in i_to_j_map:
        solver.prob += (pulp.lpSum(y[(i, j)] for j in i_to_j_map[i]) == 1, f"Must_Serve_{i}")

# 4.2 Big-M & 物理 & 政策
for (i, j) in y_keys:
    # Logic: y <= z (M=1)
    solver.prob += (y[(i,j)] <= z[j], f"BigM_{i}_{j}")

for idx, row in df_sites.iterrows():
    j = row['Site_ID']
    cap = row['Grid_Capacity_kW']
    
    # 物理限制
    solver.prob += (n_fast[j] + n_slow[j] <= 50 * z[j], f"Phy_{j}")
    # 电网限制
    solver.prob += (120 * n_fast[j] + 7 * n_slow[j] <= cap, f"Grid_{j}")
    # 商业区政策 (假设 df_sites 已有 Zone_Type 列)
    if row.get('Zone_Type') == 'Commercial':
        solver.prob += (n_fast[j] >= n_slow[j], f"Pol_{j}")

# 4.3 供需平衡 (稍微放宽系数防死锁: 3 -> 3.5)
traffic_map = dict(zip(df_demands_clean['Demand_ID'], df_demands_clean['Daily_Traffic']))
j_to_i_map = {}
for (i, j) in y_keys:
    if j not in j_to_i_map: j_to_i_map[j] = []
    j_to_i_map[j].append(i)

for j in sites_ids:
    if j in j_to_i_map:
        load = pulp.lpSum(y[(i,j)] * traffic_map[i] for i in j_to_i_map[j])
        cap = 3.5 * (n_fast[j] + n_slow[j]) # Relaxed coef
        solver.prob += (cap >= load, f"Bal_{j}")

# 4.4 目标函数
land_cost_map = dict(zip(df_sites['Site_ID'], df_sites.get('Land_Cost', 100)))
obj = (pulp.lpSum(z[j] * land_cost_map[j] for j in sites_ids) + 
       pulp.lpSum(n_fast[j]*10 + n_slow[j]*2 for j in sites_ids) +
       pulp.lpSum(valid_dist_dict[k] * y[k] * 0.1 for k in y_keys))
solver.prob += obj

# 5. Phase 3: 再次求解
# ------------------------------------------
print(f"\n🚀 [Re-Solving] 启动求解器 (Time=60s)...")
solver.solve(time_limit=60, gap_rel=0.05)

if solver.prob.status == 1:
    print(f"\n🏆 [GreenGrid 修正后方案]")
    plan = []
    for j in sites_ids:
        if pulp.value(z[j]) > 0.5:
            # 统计服务流量
            served = sum(traffic_map[i] for i in j_to_i_map.get(j, []) if pulp.value(y[(i,j)]) > 0.5)
            plan.append({
                'Site': j,
                'Zone': df_sites.loc[df_sites['Site_ID']==j, 'Zone_Type'].values[0],
                'Fast': int(pulp.value(n_fast[j])),
                'Slow': int(pulp.value(n_slow[j])),
                'Traffic': round(served, 1)
            })
    print(pd.DataFrame(plan).to_string(index=False))
else:
    print("❌ 依然无解。建议进一步检查电网容量限制。")


🔄 [System Reboot] 正在应用修复补丁 (Threshold=35km)...
✅ 空间索引重构完成: 337 valid pairs (Threshold=35)
✅ 孤岛危机解除: 所有需求点均已覆盖。

🚀 [Re-Solving] 启动求解器 (Time=60s)...

🚀 启动 MIP 求解器 (CBC)...
   -> 限制: Time < 60s, Gap < 5.0%
📋 求解状态: Infeasible
🚨 警告: 模型无解或无界！启动自动诊断...

🔍 --- MIP 诊断建议 ---
1. **互斥检查**: sum(x) == 1 是否因资源不足无法满足？
2. **Big-M**: M 是否过小截断了可行域？
3. **建议**: 尝试将 Integer 改为 Continuous 求解，定位是否为整数性导致的无解。
❌ 依然无解。建议进一步检查电网容量限制。


In [10]:
# ==========================================
# 🛡️ Phase 3.6: Capacity Scan & Safe Mode
# ==========================================

print(f"\n📊 [Deep Scan] 正在扫描全网容量瓶颈...")

# 1. 宏观容量检查 (Macro Check)
# ------------------------------------------
total_daily_traffic = df_demands_clean['Daily_Traffic'].sum()

# 计算理论最大服务能力 (假设全建慢充，这是最省电的情况)
# Max_N_Slow = Grid_Capacity / 7
# Max_Service = Max_N_Slow * 3.5 (放松后的系数)
total_theoretical_capacity = 0
for idx, row in df_sites.iterrows():
    max_slots = 50 # 物理上限
    max_power = row['Grid_Capacity_kW'] / 7 # 电力上限
    real_max_n = min(max_slots, max_power)
    total_theoretical_capacity += real_max_n * 3.5

load_ratio = total_daily_traffic / total_theoretical_capacity if total_theoretical_capacity else 999
print(f"   - 全网总需求 (Traffic): {total_daily_traffic}")
print(f"   - 全网理论极限容量 (Max Cap): {total_theoretical_capacity:.1f}")
print(f"   - 负载率 (Load Ratio): {load_ratio:.2%}")

if load_ratio > 1.0:
    print("❌ [致命] 全网总容量不足以支撑需求！必须增加 Grid_Capacity 或 削减 Traffic。")
elif load_ratio > 0.8:
    print("⚠️ [警告] 系统负载极高 (>80%)，'商业区快充策略' 极大概率会导致无解。")
else:
    print("✅ [通过] 总容量充足，问题出在局部约束或策略冲突。")

# 2. 安全模式求解 (Safe Mode Solve)
# ------------------------------------------
print(f"\n🏳️ [Safe Mode] 启动降级求解 (禁用商业策略 + 极限服务系数)...")

# 修改约束 (在原 solver 对象上修改是复杂的，建议快速重建核心约束)
# 为了演示，我们直接清除原问题中的冲突约束
# 注意: Pulp 不支持直接删除单条约束，我们这里重新创建一个仅包含核心逻辑的 clean_solver

safe_solver = IP_Solver_Capsule(name="GreenGrid_SafeMode", sense='min')
# 复用变量
safe_solver.single_vars = solver.single_vars
safe_solver.matrix_vars = solver.matrix_vars
safe_solver.binary_vars = solver.binary_vars
# 重新注入 y 到 safe_solver (pulp 变量绑定到了原 prob，这里其实需要完全重建，
# 但为了代码简洁，我们尝试只添加核心约束。如果报错，说明需完全重实例化)

# --- 简易重建流程 (Fast Track) ---
x = safe_solver.prob # alias
# A. 刚性服务 (保留)
for i in all_demands:
    if i in i_to_j_map:
        x += (pulp.lpSum(y[(i, j)] for j in i_to_j_map[i]) == 1, f"Safe_Serve_{i}")

# B. 核心物理限制 (保留)
for idx, row in df_sites.iterrows():
    j = row['Site_ID']
    cap = row['Grid_Capacity_kW']
    # 物理位
    x += (n_fast[j] + n_slow[j] <= 50 * z[j], f"Safe_Phy_{j}")
    # 电网 (关键瓶颈)
    x += (120 * n_fast[j] + 7 * n_slow[j] <= cap, f"Safe_Grid_{j}")
    # Big-M
    for i_demand in j_to_i_map.get(j, []):
        x += (y[(i_demand, j)] <= z[j], f"Safe_Logic_{i_demand}_{j}")

# C. 供需平衡 (超级放松: 系数改为 5.0, 仅用于测试可行性)
# 意味着 1个桩能服务 5个单位流量 (原为3.0)
relaxed_coef = 5.0
for j in sites_ids:
    if j in j_to_i_map:
        load = pulp.lpSum(y[(i,j)] * traffic_map[i] for i in j_to_i_map[j])
        x += (relaxed_coef * (n_fast[j] + n_slow[j]) >= load, f"Safe_Bal_{j}")

# D. 屏蔽商业策略 (❌ 已移除 n_fast >= n_slow)

# E. 目标函数
x += obj

# 求解
safe_solver.solve(time_limit=60, gap_rel=0.05)

if safe_solver.prob.status == 1:
    print(f"\n🎉 [Safe Mode Success] 找到可行解! (Service Coef={relaxed_coef}, No Policy)")
    print("💡 结论: 原模型无解是因为 '服务效率系数(3.0)' 太低 或 '商业策略' 过于激进。")
    print("🛠️ 建议: 在最终模型中，请尝试将服务系数调整为 4.0 或 5.0，或者允许商业区多建慢充。")
    
    # 简单打印
    for j in sites_ids[:5]: # 打印前5个看看
         if pulp.value(z[j]) > 0.5:
            print(f"Site {j}: Fast={pulp.value(n_fast[j])}, Slow={pulp.value(n_slow[j])}")
else:
    print(f"💀 [Critical] 安全模式依然无解。请检查: 数据中是否存在 Traffic > 500 但周围 Site Capacity < 100 的情况？")


📊 [Deep Scan] 正在扫描全网容量瓶颈...
   - 全网总需求 (Traffic): 4195.0
   - 全网理论极限容量 (Max Cap): 3355.0
   - 负载率 (Load Ratio): 125.04%
❌ [致命] 全网总容量不足以支撑需求！必须增加 Grid_Capacity 或 削减 Traffic。

🏳️ [Safe Mode] 启动降级求解 (禁用商业策略 + 极限服务系数)...

🚀 启动 MIP 求解器 (CBC)...
   -> 限制: Time < 60s, Gap < 5.0%
📋 求解状态: Optimal
💎 最优目标值: 3647.349418396102

🎉 [Safe Mode Success] 找到可行解! (Service Coef=5.0, No Policy)
💡 结论: 原模型无解是因为 '服务效率系数(3.0)' 太低 或 '商业策略' 过于激进。
🛠️ 建议: 在最终模型中，请尝试将服务系数调整为 4.0 或 5.0，或者允许商业区多建慢充。
Site S01: Fast=0.0, Slow=41.0
Site S02: Fast=0.0, Slow=34.0
Site S03: Fast=1.0, Slow=44.0
Site S04: Fast=0.0, Slow=49.0
Site S05: Fast=0.0, Slow=50.0


In [11]:
# ==========================================
# 🛑 Phase 4: 紧急扩容与最终求解 (The Final Solution)
# ==========================================
import pulp
import pandas as pd
import numpy as np

print(f"\n⚡ [Emergency Action] 正在执行电网扩容协议...")

# 1. 数据修正: 强制扩容 50%
# ------------------------------------------
# 原地修改 df_sites，提升容量以满足需求
df_sites['Grid_Capacity_kW'] = df_sites['Grid_Capacity_kW'] * 1.5
new_total_cap = 0
for _, row in df_sites.iterrows():
    # 重新计算理论容量 (按全慢充估算)
    max_slots = 50
    max_power = row['Grid_Capacity_kW'] / 7
    new_total_cap += min(max_slots, max_power) * 3.5 # 系数 3.5

print(f"   - 扩容后全网极限容量: {new_total_cap:.1f} (vs Demand: 4195.0)")
print(f"   - 新负载率: {4195 / new_total_cap:.2%}")
if new_total_cap < 4195:
    print("⚠️ 警告: 扩容后依然紧张，可能需要进一步放宽服务系数。")
else:
    print("✅ 容量充足。模型现在具有物理可行性。")

# 2. 模型完全重构 (Final Build)
# ------------------------------------------
solver = IP_Solver_Capsule(name="GreenGrid_Final", sense='min')

# 变量重建 (复用 Phase 0 的 valid_pairs)
# ------------------------------------------
sites_ids = df_sites['Site_ID'].tolist()
y_keys = [(p[0], p[1]) for p in valid_pairs] # 确保 valid_pairs 存在

z = pulp.LpVariable.dicts("z", sites_ids, cat='Binary')
n_fast = pulp.LpVariable.dicts("n_fast", sites_ids, lowBound=0, upBound=50, cat='Integer')
n_slow = pulp.LpVariable.dicts("n_slow", sites_ids, lowBound=0, upBound=50, cat='Integer')
y = pulp.LpVariable.dicts("y", y_keys, cat='Binary')

# 注册变量以便导出
solver.single_vars.update({'z': z, 'n_fast': n_fast, 'n_slow': n_slow})
solver.matrix_vars['y'] = y
solver.binary_vars.extend(z.values())
solver.binary_vars.extend(y.values())

# 约束重注 (Strict Logic)
# ------------------------------------------
# A. 刚性服务 (Must Serve)
i_to_j_map = {}
for (i, j) in y_keys:
    if i not in i_to_j_map: i_to_j_map[i] = []
    i_to_j_map[i].append(j)

demand_ids = df_demands_clean['Demand_ID'].unique()
for i in demand_ids:
    if i in i_to_j_map:
        solver.prob += (pulp.lpSum(y[(i, j)] for j in i_to_j_map[i]) == 1, f"Final_Serve_{i}")

# B. 物理与政策
for idx, row in df_sites.iterrows():
    j = row['Site_ID']
    cap = row['Grid_Capacity_kW']
    
    # 物理位 & 电网硬限
    solver.prob += (n_fast[j] + n_slow[j] <= 50 * z[j], f"Final_Phy_{j}")
    solver.prob += (120 * n_fast[j] + 7 * n_slow[j] <= cap, f"Final_Grid_{j}")
    
    # Big-M (y <= z)
    for i_demand in j_to_i_map.get(j, []):
        solver.prob += (y[(i_demand, j)] <= z[j], f"Final_Logic_{i_demand}_{j}")
    
    # 商业区政策 (尝试保留，如果容量够大应该能解)
    if row.get('Zone_Type') == 'Commercial':
        solver.prob += (n_fast[j] >= n_slow[j], f"Final_Policy_{j}")

# C. 供需平衡 (使用标准系数 3.5)
traffic_map = dict(zip(df_demands_clean['Demand_ID'], df_demands_clean['Daily_Traffic']))
service_coef = 3.5 

for j in sites_ids:
    if j in i_to_j_map.values(): # 只要有连接
         # 注意: i_to_j_map 是 i->[j], 这里我们需要反向 j->[i]
         pass 

# 构建 j->i 映射
j_to_i_map = {}
for (i, j) in y_keys:
    if j not in j_to_i_map: j_to_i_map[j] = []
    j_to_i_map[j].append(i)

for j in sites_ids:
    if j in j_to_i_map:
        load = pulp.lpSum(y[(i,j)] * traffic_map[i] for i in j_to_i_map[j])
        cap_service = service_coef * (n_fast[j] + n_slow[j])
        solver.prob += (cap_service >= load, f"Final_Bal_{j}")

# D. 目标函数
land_cost_map = dict(zip(df_sites['Site_ID'], df_sites.get('Land_Cost', 100)))
obj = (pulp.lpSum(z[j] * land_cost_map[j] for j in sites_ids) + 
       pulp.lpSum(n_fast[j]*10 + n_slow[j]*2 for j in sites_ids) +
       pulp.lpSum(valid_dist_dict[k] * y[k] * 0.1 for k in y_keys))
solver.prob += obj

# 3. 求解与交付
# ------------------------------------------
print(f"\n🚀 [Final Launch] 启动最终求解 (Capacity Expanded)...")
solver.solve(time_limit=60, gap_rel=0.05)

if solver.prob.status == 1:
    print(f"\n🏆 [GreenGrid 扩容版建设方案]")
    plan = []
    for j in sites_ids:
        if pulp.value(z[j]) > 0.5:
            served = sum(traffic_map[i] for i in j_to_i_map.get(j, []) if pulp.value(y[(i,j)]) > 0.5)
            plan.append({
                'Site': j,
                'Zone': df_sites.loc[df_sites['Site_ID']==j, 'Zone_Type'].values[0],
                'Cap(Old)': round(row['Grid_Capacity_kW']/1.5, 1), # 还原显示
                'Cap(New)': round(df_sites.loc[df_sites['Site_ID']==j, 'Grid_Capacity_kW'].values[0], 1),
                'Fast': int(pulp.value(n_fast[j])),
                'Slow': int(pulp.value(n_slow[j])),
                'Traffic': round(served, 1)
            })
    df_res = pd.DataFrame(plan)
    print(df_res.to_string(index=False))
    
    # 触发 Phase 5 可视化接口
    print("\n🗺️ [Visualization Ready]")
    print("   -> 请调用: solver.visualize_routing(...) 查看扩容后的网络图")
else:
    print("❌ 依然无解。这说明商业区快充策略与扩容后的容量仍有冲突，建议彻底废除商业策略。")


⚡ [Emergency Action] 正在执行电网扩容协议...
   - 扩容后全网极限容量: 3500.0 (vs Demand: 4195.0)
   - 新负载率: 119.86%
⚠️ 警告: 扩容后依然紧张，可能需要进一步放宽服务系数。

🚀 [Final Launch] 启动最终求解 (Capacity Expanded)...

🚀 启动 MIP 求解器 (CBC)...
   -> 限制: Time < 60s, Gap < 5.0%
📋 求解状态: Infeasible
🚨 警告: 模型无解或无界！启动自动诊断...

🔍 --- MIP 诊断建议 ---
1. **互斥检查**: sum(x) == 1 是否因资源不足无法满足？
2. **Big-M**: M 是否过小截断了可行域？
3. **建议**: 尝试将 Integer 改为 Continuous 求解，定位是否为整数性导致的无解。
❌ 依然无解。这说明商业区快充策略与扩容后的容量仍有冲突，建议彻底废除商业策略。


In [12]:
# ==========================================
# 🏁 Phase 5: 终极求解 (The Ultimate Solution)
# ==========================================
import pulp
import pandas as pd
import numpy as np

print(f"\n☢️ [Final Protocol] 启动终极妥协方案...")

# 1. 暴力扩容 (x3.0)
# ------------------------------------------
# 重置容量为原始值的 3 倍
df_sites['Grid_Capacity_kW'] = df_sites['Grid_Capacity_kW'] / 1.5 * 3.0  # 还原后乘3
new_total_cap = 0
for _, row in df_sites.iterrows():
    # 按全慢充计算极限容量
    cap_kw = row['Grid_Capacity_kW']
    max_slots = 50
    max_power_slots = cap_kw / 7
    new_total_cap += min(max_slots, max_power_slots) * 4.0 # 系数 4.0

print(f"   - [扩容] 扩容系数: 3.0x")
print(f"   - [容量] 全网极限容量: {new_total_cap:.1f} (vs Demand: 4195.0)")
print(f"   - [策略] 商业区快充优先策略: ❌ 已废除")
print(f"   - [参数] 服务系数 (Service Coef): 3.5 -> 4.0")

# 2. 重建模型 (Clean Build)
# ------------------------------------------
solver = IP_Solver_Capsule(name="GreenGrid_Ultimate", sense='min')

# 变量重建
sites_ids = df_sites['Site_ID'].tolist()
# 确保 valid_pairs 存在 (从之前步骤继承)
y_keys = [(p[0], p[1]) for p in valid_pairs] 

z = pulp.LpVariable.dicts("z", sites_ids, cat='Binary')
n_fast = pulp.LpVariable.dicts("n_fast", sites_ids, lowBound=0, upBound=50, cat='Integer')
n_slow = pulp.LpVariable.dicts("n_slow", sites_ids, lowBound=0, upBound=50, cat='Integer')
y = pulp.LpVariable.dicts("y", y_keys, cat='Binary')

# 注册变量
solver.single_vars.update({'z': z, 'n_fast': n_fast, 'n_slow': n_slow})
solver.matrix_vars['y'] = y
solver.binary_vars.extend(z.values())
solver.binary_vars.extend(y.values())

# 3. 约束注入 (Simplified Logic)
# ------------------------------------------
# A. 刚性服务
i_to_j_map = {}
for (i, j) in y_keys:
    if i not in i_to_j_map: i_to_j_map[i] = []
    i_to_j_map[i].append(j)

demand_ids = df_demands_clean['Demand_ID'].unique()
for i in demand_ids:
    if i in i_to_j_map:
        solver.prob += (pulp.lpSum(y[(i, j)] for j in i_to_j_map[i]) == 1, f"Ult_Serve_{i}")

# B. 物理与电网 (保留)
for idx, row in df_sites.iterrows():
    j = row['Site_ID']
    cap = row['Grid_Capacity_kW']
    
    # 物理位
    solver.prob += (n_fast[j] + n_slow[j] <= 50 * z[j], f"Ult_Phy_{j}")
    # 电网硬限
    solver.prob += (120 * n_fast[j] + 7 * n_slow[j] <= cap, f"Ult_Grid_{j}")
    # Big-M
    for i_demand in j_to_i_map.get(j, []):
        solver.prob += (y[(i_demand, j)] <= z[j], f"Ult_Logic_{i_demand}_{j}")
    
    # [关键改变] 移除了商业区 n_fast >= n_slow 约束

# C. 供需平衡 (系数 4.0)
traffic_map = dict(zip(df_demands_clean['Demand_ID'], df_demands_clean['Daily_Traffic']))
service_coef = 4.0 # Relaxed

# 构建 j->i 映射
j_to_i_map = {}
for (i, j) in y_keys:
    if j not in j_to_i_map: j_to_i_map[j] = []
    j_to_i_map[j].append(i)

for j in sites_ids:
    if j in j_to_i_map:
        load = pulp.lpSum(y[(i,j)] * traffic_map[i] for i in j_to_i_map[j])
        cap_service = service_coef * (n_fast[j] + n_slow[j])
        solver.prob += (cap_service >= load, f"Ult_Bal_{j}")

# D. 目标函数
land_cost_map = dict(zip(df_sites['Site_ID'], df_sites.get('Land_Cost', 100)))
obj = (pulp.lpSum(z[j] * land_cost_map[j] for j in sites_ids) + 
       pulp.lpSum(n_fast[j]*10 + n_slow[j]*2 for j in sites_ids) +
       pulp.lpSum(valid_dist_dict[k] * y[k] * 0.1 for k in y_keys))
solver.prob += obj

# 4. 求解与交付
# ------------------------------------------
print(f"\n🚀 [Ultimate Launch] 启动最终求解...")
solver.solve(time_limit=60, gap_rel=0.05)

if solver.prob.status == 1:
    print(f"\n🏆 [GreenGrid 终极建设方案]")
    plan = []
    for j in sites_ids:
        if pulp.value(z[j]) > 0.5:
            served = sum(traffic_map[i] for i in j_to_i_map.get(j, []) if pulp.value(y[(i,j)]) > 0.5)
            plan.append({
                'Site': j,
                'Zone': df_sites.loc[df_sites['Site_ID']==j, 'Zone_Type'].values[0],
                'Cap(New)': int(df_sites.loc[df_sites['Site_ID']==j, 'Grid_Capacity_kW'].values[0]),
                'Fast': int(pulp.value(n_fast[j])),
                'Slow': int(pulp.value(n_slow[j])),
                'Traffic': round(served, 1)
            })
    df_res = pd.DataFrame(plan)
    # 按流量降序
    df_res = df_res.sort_values('Traffic', ascending=False)
    print(df_res.to_string(index=False))
    
    # 导出结果
    solver.export_results()
    
    # 可视化路由
    try:
        solver.export_routing_gephi(matrix_name='y')
        # 构建 pos 字典用于绘图
        pos_dict = {row['Site_ID']: (row['X'], row['Y']) for _, row in df_sites.iterrows()}
        pos_dict.update({row['Demand_ID']: (row['X'], row['Y']) for _, row in df_demands_clean.iterrows()})
        
        # 提取被选中的 Site ID
        active_sites = [r['Site'] for r in plan]
        solver.visualize_routing(from_nodes=df_demands_clean['Demand_ID'], 
                                 to_nodes=active_sites, 
                                 active_matrix_name='y',
                                 pos_dict=pos_dict)
        print("\n🗺️ 可视化已生成: Routing_Graph.svg (请在本地查看)")
        print("🕸️ Gephi 数据已生成: Routing_Flow_Gephi.csv")
    except Exception as e:
        print(f"⚠️ 可视化生成失败: {e}")

else:
    print("💀 依然无解。建议检查数据文件是否存在逻辑矛盾（如某些点距离所有站点都 > 35km）。")


☢️ [Final Protocol] 启动终极妥协方案...
   - [扩容] 扩容系数: 3.0x
   - [容量] 全网极限容量: 4000.0 (vs Demand: 4195.0)
   - [策略] 商业区快充优先策略: ❌ 已废除
   - [参数] 服务系数 (Service Coef): 3.5 -> 4.0

🚀 [Ultimate Launch] 启动最终求解...

🚀 启动 MIP 求解器 (CBC)...
   -> 限制: Time < 60s, Gap < 5.0%
📋 求解状态: Infeasible
🚨 警告: 模型无解或无界！启动自动诊断...

🔍 --- MIP 诊断建议 ---
1. **互斥检查**: sum(x) == 1 是否因资源不足无法满足？
2. **Big-M**: M 是否过小截断了可行域？
3. **建议**: 尝试将 Integer 改为 Continuous 求解，定位是否为整数性导致的无解。
💀 依然无解。建议检查数据文件是否存在逻辑矛盾（如某些点距离所有站点都 > 35km）。
